In [70]:
import numpy as np
import pandas as pd
import nltk.sentiment.sentiment_analyzer
import gzip 

In [71]:
def parse(path): 
    v = gzip.open(path, 'rb') 
    for b in v: 
        yield eval(b) 

In [72]:
def Df(path): 
    idx = 0 
    d = {} 
    for f in parse(path): 
        d[idx] = f
        idx += 1 
    return pd.DataFrame.from_dict(d, orient='index') 

In [73]:
amz = Df('reviews_Musical_Instruments_5.json.gz')

In [86]:
negativesentence = open("negative.txt").read()
neg_words=negsentence.split('\n')

In [75]:
positivesentence = open("positive.txt").read()
pos_words=positivesentence.split('\n')

In [76]:
df = pd.DataFrame(amz['reviewText'])
df['positive_count'] = 0
df['negative_count'] = 0
df['negator_count'] = 0
df['tags'] = amz['overall'].astype(int)

In [77]:
for i in range(len(df)):
    pos_cnt = 0
    neg_cnt = 0
    negator_cnt = 0
    text = df.iloc[i, 0]
    words = text.split(' ')
    for word in words:
        if word in pos_words:
            pos_cnt = pos_cnt + 1
        if word in neg_words:
            neg_cnt = neg_cnt + 1
    res = nltk.sentiment.util.mark_negation(text.split()) 
    for word in res: 
        if word.endswith('_NEG'):
            negator_cnt = negator_cnt + 1
    df.iloc[i, 1] = pos_count
    df.iloc[i, 2] = neg_count
    df.iloc[i, 3] = negator_count

In [78]:
X = df.iloc[:, 1:4]    
Y = df.iloc[:, 4]

In [79]:
from sklearn import preprocessing
scalar = preprocessing.StandardScaler()
X = scalar.fit_transform(X)

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [81]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial', solver='sag', max_iter = 5000)
lr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [82]:
Y_pred = clf.predict(X_test)

In [83]:
print("Accuracy on testing set: ", np.mean(Y_pred == Y_test))

Accuracy on testing set:  0.6692644909887969


In [84]:
print("Accuracy on whole Dataset: ", clf.score(X, Y))

Accuracy on whole Dataset:  0.6761524217912485


In [85]:
from sklearn.metrics import confusion_matrix
con_mat = confusion_matrix(Y_test, Y_pred)
print("Confusion matrix: \n", con_mat)

Confusion matrix: 
 [[   0    0    0    0   39]
 [   0    0    0    0   55]
 [   0    0    0    0  134]
 [   0    0    0    0  451]
 [   0    0    0    0 1374]]
